## Model generator main file

In [12]:
import numpy as np
import re
import unidecode
import os
import pickle
import sys

import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.model_selection     import GridSearchCV

import keras.backend as K
from keras.wrappers.scikit_learn import KerasClassifier
from keras.constraints           import maxnorm
from keras                       import regularizers
from keras                       import Input, Model, Sequential
from keras.layers                import CuDNNLSTM, Bidirectional, Activation, TimeDistributed, Dense, RepeatVector, Embedding, Dropout, BatchNormalization
from keras.layers.recurrent      import LSTM, GRU, SimpleRNN
from keras.utils                 import np_utils
from keras.callbacks             import EarlyStopping, TensorBoard, ModelCheckpoint

Import data and make folders

In [13]:
data_path = 'data'
temp_path = data_path + '/temp'
backup_path = data_path + '/backup'
weights_path = data_path + '/weights'
train_history = data_path + '/train_history'

if not os.path.isdir(data_path):
    os.mkdir(data_path)
if not os.path.isdir(temp_path):
    os.mkdir(temp_path)
if not os.path.isdir(backup_path):
    os.mkdir(backup_path)
if not os.path.isdir(weights_path):
    os.mkdir(weights_path)
if not os.path.isdir(train_history):
    os.mkdir(train_history)

#joint_angle_data.pickle
with open(data_path + '/augmented_input.pickle', 'rb') as file:
    joint_angle_data = pickle.load(file)
    file.close()

#power_data.pickle
with open(data_path + '/filtered_power_data.pickle', 'rb') as file:
    power_data = pickle.load(file)
    file.close()
  

## Recurrent neural networks

Different model functions

In [14]:
def mean_square_error(y_true, y_pred):
    return K.mean(K.square(y_true-y_pred))

In [15]:
def setModelToBasicLSTM(self, multiple=True):
    n_timesteps, n_features, n_outputs = self.X_train.shape[1], self.X_train.shape[2], self.Y_train.shape[1]
    # define model
    self.model = Sequential()
    self.model.add(Bidirectional(CuDNNLSTM(128, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero', input_shape = (n_timesteps, n_features))))
    self.model.add(Bidirectional(CuDNNLSTM(128, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Bidirectional(CuDNNLSTM(128, return_sequences = True, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Bidirectional(CuDNNLSTM(128, return_sequences = multiple, kernel_initializer = 'random_uniform', bias_initializer = 'zero')))
    self.model.add(Dropout(0.2))
    self.model.add(Dense(64, activation='relu', kernel_initializer = 'random_uniform', bias_initializer = 'zero'))
    self.model.add(Dense(n_outputs))
    self.model.compile(loss='mean_squared_error', optimizer='adam', metrics=[mean_square_error])


In [24]:
def setModelToGridSearch(self, multiple=True, layer_type=CuDNNLSTM, hidden_units_RNN=128, hidden_units_dense=64, num_mid_layers = 3, optimizer='adam', learn_rate=0.01, momentum=0, init_mode='random_uniform', activation='relu', dropout_rate=0.0, weight_constraint=0):
    n_timesteps, n_features, n_outputs = self.X_train.shape[1], self.X_train.shape[2], self.Y_train.shape[1]
    # define model
    self.model = Sequential()
    self.model.add(Bidirectional(layer_type(hidden_units_RNN, return_sequences = True, kernel_initializer = init_mode, bias_initializer = 'zero', input_shape = (n_timesteps, n_features))))
    for n in range(num_mid_layers):
        self.model.add(Bidirectional(layer_type(hidden_units_RNN, return_sequences = True, kernel_initializer = init_mode, bias_initializer = 'zero')))
    
    self.model.add(Bidirectional(layer_type(hidden_units_RNN, return_sequences = multiple, kernel_initializer = init_mode, bias_initializer = 'zero')))
    self.model.add(Dropout(dropout_rate))
    self.model.add(Dense(hidden_units_dense, activation=activation, kernel_initializer = 'random_uniform', bias_initializer = 'zero', kernel_constraint=maxnorm(weight_constraint)))
    self.model.add(Dense(n_outputs))
    self.model.compile(loss='mean_squared_error', optimizer=optimizer, metrics=[mean_square_error])
    return self.model

Other functions

In [25]:
def series_to_supervised(dataX, dataY, n_input, n_output):
    #Output: [batchs, n_timesteps, n_features]
    df_X = pd.DataFrame(dataX)
    df_Y = pd.DataFrame(dataY)
    X = np.ndarray((df_X.shape[0]-n_input, n_input, df_X.shape[1]) )
    Y = np.ndarray((df_Y.shape[0]-n_input, n_output))

    for n in range(0,df_X.shape[0]-n_input-1,1):
        for m in range(n_input-1,-1, -1):
            X[n,m,:] = df_X.loc[n+m:n+m,:]
            
    for n in range(0,df_Y.shape[0]-n_input-1,1):
        for m in range(n_output-1,-1, -1):
            Y[n,:] = df_Y.loc[n+m:n+m,:]
            
    return X, Y

In [26]:
def setUpData(self, seq_length, dataX, dataY, n_test_ratio):

    self.X_train = []
    self.Y_train = []
    self.X_test = []
    self.Y_test = []
    self.seq_length = seq_length
    
    X, Y = series_to_supervised(dataX, dataY, n_input=seq_length, n_output=dataY.shape[1])
    
    self.X_train = X[:int(len(X)*(1-n_test_ratio)), :, :]
    self.Y_train = Y[:int(len(Y)*(1-n_test_ratio)), :]
    self.X_test = X[int(len(X)*(1-n_test_ratio)):, :, :]
    self.Y_test = Y[int(len(Y)*(1-n_test_ratio)):, :]
    

In [30]:
def trainModel(self, name):
    # File path for model
    filepath = weights_path + "/weights-" + name + "-{epoch:02d}-{loss:.4f}.hdf5"
    # Callbacks functions
    es = EarlyStopping(monitor='val_loss', patience=40, verbose=0)
    tb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32)
    mc = ModelCheckpoint(filepath, monitor='val_loss', verbose=0, save_best_only=True, mode='auto')
    
    # Train the model
    epochs = 1           #Maximum number of epochs to run
    batch_size = 16    #Size of training data batch
    Val_split = 0.1       #Procentage of training data to use as validation data
    history = self.model.fit(self.X_train, self.Y_train, epochs=epochs, batch_size=batch_size, validation_split=Val_split, callbacks=[es, tb, mc])
    
    # Save the model
    filename = "model_" + name + ".hdf5"
    self.model.save_weights(weights_path + '/' + filename)
    # Save the history
    filename = "history_" + name + ".pickle"
    with open(train_history + '/' + filename, 'wb') as f:
        pickle.dump(history.history, f)
        f.close()

Model generator

In [31]:
class Generator:

    set_up_data = setUpData
    train_model = trainModel
    
    set_model_to_Basic_LSTM = setModelToBasicLSTM
    set_model_to_Grid_Search = setModelToGridSearch
    

## Training the network

Test training

In [33]:
gen = Generator()
seqLleng = 10 #12ms/leng unit
n_test_ratio = 0.02
print((power_data.shape))
gen.set_up_data(seqLleng, joint_angle_data, power_data, n_test_ratio)
print(gen.X_train.shape)
print(gen.Y_train.shape)
print(gen.X_test.shape)
print(gen.Y_test.shape)
gen.set_model_to_Basic_LSTM(multiple=False)
name = 'test_run'
gen.train_model(name = name)

(13899, 1)
(13611, 10, 19)
(13611, 1)
(278, 10, 19)
(278, 1)
Train on 12249 samples, validate on 1362 samples
Epoch 1/1
12249/12249 [==============================] - 22s 2ms/step - loss: 10.7358 - mean_square_error: 10.7358 - val_loss: 10.9098 - val_mean_square_error: 10.9098


Hyperparamter Grid Search

In [51]:
gen = Generator()
# Variables
seqLleng = 10 #12ms/leng unit
n_test_ratio = 0.02
# Set up the data
print('Setting up data')
gen.set_up_data(seqLleng, joint_angle_data, power_data, n_test_ratio)
# create model
print('Creating keras Classifier')
model = KerasClassifier(build_fn=gen.set_model_to_Grid_Search, epochs=1, batch_size=16, verbose=1)
# define the grid search parameters
layer_type = [CuDNNLSTM]
hidden_units_RNN = [32, 64, 128]
hidden_units_dense = [64]
num_mid_layers = [1]
optimizer = ['adam']
learn_rate = [0.01]
momentum = [0]
init_mode = ['random_uniform']
activation = ['relu']
dropout_rate = [0.0]
weight_constraint = [0]
# Make dictionary
param_grid = dict(layer_type=layer_type,hidden_units_RNN=hidden_units_RNN,hidden_units_dense=hidden_units_dense,num_mid_layers=num_mid_layers)
# Grid Search
print('Grid Search Starting')
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=1)
print('X: ' + str(gen.X_train.shape))
print('Y: ' + str(gen.Y_train.shape))
grid_result = grid.fit(gen.X_train, gen.Y_train)

Setting up data
Creating keras Classifier
Grid Search Starting
X: (13611, 10, 19)
Y: (13611, 1)


ValueError: Error when checking target: expected dense_20 to have 3 dimensions, but got array with shape (9074, 1)

In [50]:
gen.set_model_to_Grid_Search()
print(gen.model.summary())

ValueError: This model has never been called, thus its weights have not yet been created, so no summary can be displayed. Build the model first (e.g. by calling it on some test data).

## Test prediction

In [ ]:
predictions = gen.model.predict(gen.X_test)

Plot the test prediction

In [ ]:
plt.figure(figsize=(12,6))
fig = plt.subplot()
x = range(len(predictions))
fig.plot(x, gen.Y_test,  label='true',  marker='o',linewidth=0.3, markersize=1.5)
fig.plot(x, predictions,  label='pred',  marker='o',linewidth=0.3, markersize=4)
fig.legend();
fig.set_ylabel('Power[W]')
fig.set_xlabel('Time')
fig.set_title('Current');